In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import os
import json
from manyqa import ManyQA
from huggingface_hub import login
from pydantic import BaseModel, Field
from datetime import datetime

In [2]:
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
NVIDIA GeForce RTX 4090


In [3]:
hf_token = os.getenv("HF_TOKEN") or os.getenv("HUGGINGFACE_HUB_TOKEN")
assert hf_token, "Ustaw zmienną środowiskową HF_TOKEN lub HUGGINGFACE_HUB_TOKEN z wartością hf_***"
login(token=hf_token, add_to_git_credential=False)

In [ ]:
# załadowanie modelu sędziego
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Uruchamiam na urządzeniu: {device}")

model_name = "speakleash/Bielik-11B-v2.6-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, 
    # load_in_8bit=True,
    # load_in_4bit=True,
    dtype=torch.bfloat16,
)

Uruchamiam na urządzeniu: cuda


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [5]:
def decode(output: list[str]) -> str:
    return output[0].split('|im_start|> assistant\n')[-1].replace('<|im_end|>', '').replace('<tool_call> ', '').replace('</tool_call>', '')

In [6]:
def generate(messages: list[str], max_tokens: int = 500, temperature: float = 0.1, top_p: float = 0.1) -> str:
    input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
    attention_mask = torch.ones_like(input_ids)
    
    model_inputs = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    
    generated_ids = model.generate(
        model_inputs, 
        attention_mask=attention_mask, 
        max_new_tokens=max_tokens, 
        do_sample=True,
        temperature=temperature,
        top_p=top_p,
    )
    decoded = tokenizer.batch_decode(generated_ids)
    return decode(decoded)

In [7]:
# test bielika
messages = [
    {"role": "system", "content": "Odpowiadaj krótko, precyzyjnie i wyłącznie w języku polskim."},
    {"role": "user", "content": "Jakie mamy pory roku w Polsce?"},
    # {"role": "assistant", "content": "W Polsce mamy 4 pory roku: wiosna, lato, jesień i zima."},
    # {"role": "user", "content": "Która jest najcieplejsza?"}
]

input_ids = tokenizer.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True)
attention_mask = torch.ones_like(input_ids)

model_inputs = input_ids.to(device)
attention_mask = attention_mask.to(device)
model.to(device)

generated_ids = model.generate(
    model_inputs, 
    attention_mask=attention_mask, 
    max_new_tokens=500, 
    do_sample=True,
    temperature=0.1,
    top_p=0.1,
)
decoded = tokenizer.batch_decode(generated_ids)
print(decode(decoded))

W Polsce występują cztery pory roku:

1. **Wiosna** (marzec, kwiecień, maj) – okres ocieplenia, rozkwit roślin.
2. **Lato** (czerwiec, lipiec, sierpień) – najcieplejsza pora roku, długie dni.
3. **Jesień** (wrzesień, październik, listopad) – ochłodzenie, opadanie liści.
4. **Zima** (grudzień, styczeń, luty) – najzimniejsza pora, możliwe opady śniegu.

Każda pora roku ma swoje charakterystyczne cechy klimatyczne i przyrodnicze. 😊


In [8]:
# model odpowiedzi
class AnswerCheck(BaseModel):
    correct: bool = Field(False, description='Flaga oznaczająca czy udzielona odpowiedź jest poprawna z kluczem')
    clarification: str = Field(..., description='Bardzo krótkie i rzeczowe wyjaśnienie decyzji')

In [ ]:
# załaduj obiekt z pytaniami
with open('QAData.json', 'r') as f:
    data = json.loads(f.read())

questions = ManyQA(**data)

In [10]:
# sprawdzenie poprawności odpowiedzi
def check_asnwer(question, model_resp) -> str:
    example = '{"correct": true, "clarification": "Odpowiedź jest poprawna, ponieważ dokładnie odpowiada kluczowi."}'
    ex1 = '{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą osobę."}'
    ex2 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę wydarzenia zamiast osoby."}'
    ex3 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę warzyw zamiast osoby (królowej Bony).}'
    ex4 = '{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę organizacji emigracyjnej (Towarzystwo Demokratyczne Polski), a nie grupy młodych artystów (Cyganeria Warszawska).}'
    ex5 = '{"correct": false, "clarification": "Niepoprawna — użytkownik nie podał wszystkich funkcji jakie pełnił Jan Zamoyski.}'
    sys_instr = f"""ROLA: Jesteś egzaminatorem testów.
    
ZADANIE: Twoim zadaniem jest określenie, czy odpowiedź użytkownika jest **merytorycznie zgodna** z podanym kluczem (poprawną odpowiedzią).

ZASADY OCENY:
1. Odpowiedź musi znaczeniowo odpowiadać kluczowi — czyli wskazywać to samo zjawisko, osobę, miejsce, wydarzenie lub pojęcie.
2. Dopuszczalne są drobne różnice językowe (np. 'powstania listopadowego' vs 'Powstanie Listopadowe').
3. Jeśli użytkownik odpowiada nie na temat lub podaje tylko coś związanego, ale nie tożsamego z kluczem — odpowiedź jest **niepoprawna**.
4. Wyjaśnienie (clarification) ma być krótkie, jednozdaniowe i rzeczowe.
5. Dopuszczalne są odpowiedzi w innych częściach mowy. Odpowiedź: 'powstania listopadowego' jest poprawna dla klucza: 'Powstanie Litopadowe'

WYJŚCIE: Zwróć WYŁĄCZNIE poprawny JSON, bez komentarzy, bez ``` i bez dodatkowego tekstu.
JSON musi być zgodny ze schematem: {json.dumps(AnswerCheck.model_json_schema(), ensure_ascii=False)}

PRZYKŁADY:

PYTANIE: Kto był przywódcą powstania listopadowego?
KLUCZ: Piotr Wysocki
ODPOWIEDŹ: Piotr Wysocki
OCENA: {ex1}

PYTANIE: Kto był przywódcą powstania listopadowego?
KLUCZ: Piotr Wysocki
ODPOWIEDŹ: powstanie listopadowe
OCENA: {ex2}

PYTANIE: Dzięki komu na polskie stoły trafiły nieznane wcześniej warzywa zwane włoszczyzną?
KLUCZ: Królowej Bonie
ODPOWIEDŹ: włoszczyzna
OCENA: {ex3}

PYTANIE: Ośmieszali stroje salonowe i konwencje obyczajowe. Ich nieoficjalnym, własnym pismem była gazeta "Nadwiślanin". Głosili idee demokratyczno-ludowe. Mowa o przedstawicielach…
KLUCZ: Cyganerii warszawskiej
ODPOWIEDŹ: Towarzystwa Demokratycznego Polskiego
OCENA: {ex4}

PYTANIE: Jaką funkcję pełnił Jan Zamoyski w Rzeczypospolitej: Hetman wielki koronny, Kanclerz wielki koronny czy Generalny starosta krakowski? 
KLUCZ: Wszystkie wymienione
ODPOWIEDŹ: Hetman wielki koronny, Kanclerz wielki koronny
OCENA: {ex5}

PYTANIE: {question.question}
KLUCZ: {question.answer}
"""
    
    t = model_resp.split('"answer": ')[-1].split(', "')[0]
    
    user_instr = f"""ODPOWIEDŹ: {t}"""
    messages = [
            {'role': 'system', 'content': sys_instr},
            {'role': 'user', 'content': user_instr}
    ]
    return generate(messages)

In [11]:
def prepare(model_response):
    if '\\\"answer\\\": ' in model_response:
        raw = model_response.split('\\\"answer\\\": ')[-1].split('}\", ')[0].replace('\\n', '')
    else:
        raw = 'Format niepoprawny'
    for_json = '\"{' + '\\\"answer\\\": ' + raw + '}\"'
    try:
        for_json = json.loads(for_json)
    except Exception as e:
        return raw
    return for_json

In [12]:
with open('0experiment_pllum.jsonl', 'r') as file:
    responses = file.readlines()

In [13]:
responses[103]

'{"datetime": "2025-10-25 18:00:13.588011", "question": "Kiedy miała miejsce opracja wojskowa znana jako Bunt Żeligowskiego?", "model_response": "Odpowiedź: w październiku 1920 roku.", "correctness": "{\\"correct\\": true, \\"clarification\\": \\"Odpowiedź jest poprawna, ponieważ dokładnie odpowiada kluczowi.\\"}", "time_to_answer": 0.8832398119993741}\n'

In [13]:
# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

In [14]:
for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")
    # break

ID: 0; 2025-10-26 06:53:52.448639
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "francuski", "confidence": 1}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 06:53:54.734849
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "Wazów",    "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą dynastię."}
Aktualny wynik: 2/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 06:53:56.316184
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Michał Mosiołek",    "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jes

In [ ]:
# PLLuM - 448/570 -> 78,5%
# CYFRAGOVPL/PLLuM-12B-instruct

In [15]:
with open('0experiment_mistral.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")
    # break

ID: 0; 2025-10-26 07:21:08.417250
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "Francuski", "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 07:21:10.712171
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "Wiedzibończycy", "confidence": 1}
{"correct": false, "clarification": "Niepoprawna — użytkownik podał nazwę miejscowości (Wiedzibończycy) zamiast dynastii (Wazowie)."}
Aktualny wynik: 1/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 07:21:12.753841
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Aleksander Kakowski", "confidence": 1.0}
{"correct"

In [ ]:
# mistral 194/570 -> 34%
# mistralai/Mistral-7B-Instruct-v0.3

In [12]:
with open('0experiment_3_45_bielik.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")

ID: 0; 2025-10-26 11:20:05.334544
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "francuskim",  "confidence": 0.95}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 11:20:07.603861
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "wazówów",  "confidence": 1.0}
{"correct": false, "clarification": "Niepoprawna — użytkownik podał błędną formę dynastii (Wazów zamiast Wazówów)."}
Aktualny wynik: 1/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 11:20:09.447095
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Michał Mosiołek",  "confidence": 1.0}
{"correct": true, "clarificatio

In [ ]:
# bielik 3 4.5B 324/570 -> 57%
# speakleash/Bielik-4.5B-v3.0-Instruct

In [13]:
with open('0experiment_bielik.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")

ID: 0; 2025-10-26 11:43:10.501177
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "francuski", "confidence": 1}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 11:43:12.765399
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "Wazów", "confidence": 1}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą dynastię."}
Aktualny wynik: 2/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 11:43:14.418847
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Michał Mosiołek", "confidence": 1}
{"correct": true, "clarification": "Odpowiedź jest poprawna

In [ ]:
# bielik 2 11B 484/570 - 85%
# speakleash/Bielik-11B-v2.6-Instruct

In [12]:
with open('0experiment_2_5_quen.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")

ID: 0; 2025-10-26 14:44:38.848132
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "francuski", "confidence": 0.95}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 14:44:40.958180
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "szwedzki", "confidence": 0.95}
{"correct": false, "clarification": "Niepoprawna — użytkownik podał przymiotnik zamiast nazwy dynastii (Wazowie)."}
Aktualny wynik: 1/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 14:44:42.550161
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Michał Mosiołek", "confidence": 0.95}
{"correct": true, "clarification"

In [ ]:
# Qwen 2.5 14B 223/570 - 39%
# Qwen/Qwen2.5-14B-Instruct

In [13]:
def prepare(model_response):
    if '\\\"answer\\\": ' in model_response:
        raw = model_response.split('\\\"answer\\\": ')[-1].split('}')[0]
    else:
        raw = 'Format niepoprawny'
    for_json = '\"{' + '\\\"answer\\\": ' + raw + '}\"'
    try:
        for_json = json.loads(for_json)
    except Exception as e:
        return raw
    return for_json

In [12]:
with open('0experiment_gemma3.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")

ID: 0; 2025-10-26 17:59:38.984606
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "Francuski", "confidence": 0.95}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwy język urzędowy Księstwa Warszawskiego."}
Aktualny wynik: 1/1 odpowiedzi poprawnych.


ID: 1; 2025-10-26 17:59:41.170335
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "Wazów", "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą dynastię."}
Aktualny wynik: 2/2 odpowiedzi poprawnych.


ID: 2; 2025-10-26 17:59:42.694510
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Aleksander Kakowski", "confidence": 1.0}
{"correct": false, "clarification": "Niepoprawna

In [ ]:
# gemma3 12B 282/570 -> 49%
# google/gemma-3-12b-it

In [14]:
with open('0experiment_gemma3_27b.jsonl', 'r') as file:
    responses = file.readlines()

# przygotowanie liczników
correct_q, all_q = 0, 0
difficult_questions = []

for idx, resp in enumerate(responses):
    all_q += 1
    print(f'ID: {idx}; '+ str(datetime.now()))
    question = questions.qa[idx]
    # question = questions.qa[50]
    print(f"Pytanie: {question.question}")
    print(f"Klucz odp.: {question.answer}")
    prepared_resp = prepare(resp)
    print(f"Odp.: {prepared_resp}")
    checked = check_asnwer(question, prepared_resp)
    if '"correct": true' in checked:
        correct_q+=1
    else:
        difficult_questions.append(question)
    print(checked)
    print(f"Aktualny wynik: {correct_q}/{all_q} odpowiedzi poprawnych.\n\n")

ID: 0; 2025-10-28 04:06:36.384953
Pytanie: Jakim językiem urzędowym - poza polskim - posługiwano się na terenie Księstwa Warszawskiego?
Klucz odp.: Francuskim
Odp.: {"answer": "rosyjski", "confidence": 0.95}
{"correct": false, "clarification": "Niepoprawna — użytkownik podał język rosyjski, a klucz to francuski."}
Aktualny wynik: 0/1 odpowiedzi poprawnych.


ID: 1; 2025-10-28 04:06:38.330375
Pytanie: Z jakiej dynastii wywodził się Władysław IV?
Klucz odp.: Z Wazów
Odp.: {"answer": "Wazów", "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wskazał właściwą dynastię."}
Aktualny wynik: 1/2 odpowiedzi poprawnych.


ID: 2; 2025-10-28 04:06:39.951767
Pytanie: Która z tych osób: Zdzisław Lubomirski, Michał Mosiołek, Aleksander Kakowski nie wchodziła w skład Rady Regencyjnej Królestwa Polskiego?
Klucz odp.: Michał Mosiołek
Odp.: {"answer": "Michał Mosiołek", "confidence": 1.0}
{"correct": true, "clarification": "Odpowiedź jest poprawna — użytkownik wsk

In [ ]:
# gemma3 27B it 342/570 -> 60%
# google/gemma3-27b-it